2) Un importante servicio de monitoreo de aplicaciones cloud, sufrió un importante
incidente en las últimas semanas por lo cual está conduciendo una investigación.
Nuestro equipo participa en el análisis de forma externa para lo cual nos provee
los siguientes dataframes en csv:

- metrics.csv describiendo información de métricas obtenidas. El mismo tiene el
 formato ('client_id', 'integration_id' ,'metric', 'timestamp', 'value')
  donde la columna ‘metric’ indica el nombre de una métrica que se registra en
  un cierto momento ('timestamp') cuyo valor se guarda en la columna 'value'.

- clients.csv con información sobre el cliente, con el formato ('client_id, '
account_number', 'name') donde 'account_number' indica el número de cuenta de
cliente y 'name' el nombre del mismo.

Como miembro de nuestro equipo es necesario que obtengas las siguientes métricas para aportar insights a nuestro análisis:

a) Calcular el valor promedio de las metricas 'aws.vpc.network_out',
'aws.vpc.network_in', 'aws.vpc.network_rate' para los clientes de ‘account_number’
mayor al '25679247' devolviendo el resultado en el siguiente formato
(account_number, 'aws.vpc.network_out_mean', 'aws.vpc.network_in_mean',
'aws.vpc.network_rate_mean')(15pts)

In [127]:
import pandas as pd

metricsDf = pd.read_csv('/home/marcos/PycharmProjects/PracticaPandas/metrics.csv')
clientsDf = pd.read_csv('/home/marcos/PycharmProjects/PracticaPandas/clients.csv')

requestedAccounts = metricsDf.drop(columns={'timestamp', 'integration_id'})
requestedMetrics = ['aws.vpc.network_out', 'aws.vpc.network_in', 'aws.vpc.network_rate']
requestedAccounts = requestedAccounts[requestedAccounts['metric'].isin(requestedMetrics)]
requestedClients = clientsDf[clientsDf['account_number'] > 25679247]
requestedAccounts = requestedAccounts.merge(requestedClients[['client_id', 'account_number']], how='inner', on='client_id')
requestedAccounts.drop(columns='client_id', inplace=True)
requestedData = requestedAccounts.pivot_table(index='account_number', columns='metric',
                                                values='value', aggfunc='mean')
requestedData.rename_axis(columns=None, inplace=True)
requestedData.columns = ['aws.vpc.network_in_mean', 'aws.vpc.network_out_mean', 'aws.vpc.network_rate_mean']
columnsTitles=['aws.vpc.network_out_mean', 'aws.vpc.network_in_mean', 'aws.vpc.network_rate_mean']
clientsMeanMetrics = requestedData.reindex(columns=columnsTitles)
clientsMeanMetrics

,aws.vpc.network_out_mean,aws.vpc.network_in_mean,aws.vpc.network_rate_mean
account_number,,,
25679248,589.000000,401.500000,557.600000
25679249,485.666667,592.500000,622.400000
25679250,631.333333,610.200000,607.000000
25679251,104.000000,361.285714,529.000000
25679252,792.500000,452.000000,NaN
25679253,NaN,NaN,918.000000
25679254,649.833333,NaN,357.666667
25679255,267.000000,494.500000,501.333333
25679256,490.333333,NaN,347.000000


b) Calcular la diferencia entre el valor promedio obtenido en cada métrica
por cada cliente, y el valor promedio general de esa métrica (sin utilizar group by) (10pts)

In [128]:
meanMetrics = requestedAccounts.pivot_table(index='account_number', columns='metric',
                                                values='value', aggfunc='sum')
meanMetrics = meanMetrics.apply(func='mean', axis=0)
clientsMeanMetrics['aws.vpc.network_out_mean'] = abs(clientsMeanMetrics['aws.vpc.network_out_mean']\
                                                 - meanMetrics.loc['aws.vpc.network_out'])
clientsMeanMetrics['aws.vpc.network_in_mean'] = abs(clientsMeanMetrics['aws.vpc.network_in_mean']\
                                                 - meanMetrics.loc['aws.vpc.network_in'])
clientsMeanMetrics['aws.vpc.network_rate_mean'] = abs(clientsMeanMetrics['aws.vpc.network_rate_mean']\
                                                 - meanMetrics.loc['aws.vpc.network_rate'])

clientsMeanMetrics.columns = ['aws.vpc.network_out_diff', 'aws.vpc.network_in_diff', 'aws.vpc.network_rate_diff']
clientsMeanMetrics

,aws.vpc.network_out_diff,aws.vpc.network_in_diff,aws.vpc.network_rate_diff
account_number,,,
25679248,1263.793103,1549.500000,1502.133333
25679249,1367.126437,1358.500000,1437.333333
25679250,1221.459770,1340.800000,1452.733333
25679251,1748.793103,1589.714286,1530.733333
25679252,1060.293103,1499.000000,NaN
25679253,NaN,NaN,1141.733333
25679254,1202.959770,NaN,1702.066667
25679255,1585.793103,1456.500000,1558.400000
25679256,1362.459770,NaN,1712.733333
